In [2]:
import torch
import torchvision
import torchvision.datasets
import torchvision.transforms
import torch.utils.data
import torch.nn
import math
from torchvision import models as torch_model
import numpy as np
import torch.nn as nn
import graphviz
from torchview import draw_graph
import os
from matplotlib import pyplot as plot
import time

In [3]:
batch_size = 128
epochs = 5
learning_rate = 0.001
num_classes = 10
dir_name = os.getcwd()
acc_array = []

In [4]:
def get_norm_params(dataset):
    mean = dataset.data.mean(axis=(0,1,2))/255
    std = dataset.data.std(axis=(0,1,2))/255
    return torch.tensor(mean).reshape(3, 1, 1), torch.tensor(std).reshape(3, 1, 1)

In [5]:
def get_data_loader(transforms_params):

   train_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = True, download = True,
    transform = transforms_params
)
   test_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = False, download = True,
    transform = transforms_params
)
   train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True
  )

   test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size = batch_size, shuffle = False)
   return train_data_loader, test_data_loader

In [6]:
loss_function = torch.nn.CrossEntropyLoss()
print('Training parameters: learning rate = {}, batch size = {}, number of epochs = {}'.format(learning_rate, batch_size, epochs))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


Training parameters: learning rate = 0.001, batch size = 128, number of epochs = 5


device(type='cpu')

In [7]:
def get_accuracy(model, data_loader, device):
    tp = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n += labels.size(0)
            tp += (predicted == labels).sum()
    return tp / n

def get_loss(model, data_loader, device):
    loss = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            loss += loss_function(outputs, labels)
            n += labels.size(0)
    return loss / n

def accuracy(outputs, labels):
  _, predicted = torch.max(outputs.data, 1)
  return torch.tensor(torch.sum(predicted == labels).item() / len(predicted))

In [8]:
def train(Network, train_data_loader, test_data_loader, optimizer):
    for epoch in range(epochs):
        start = time.time()
        losses = []
        for i, (images, labels) in enumerate(train_data_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = Network(images)
            loss = loss_function(outputs, labels)
            losses.append(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        duration = time.time() - start
        print('Epoch[{}]: loss = {}, accuracy = {}, time = {}'.format(epoch, torch.stack(losses).mean(), get_accuracy(Network,train_data_loader, device), duration))
    acc = get_accuracy(Network,test_data_loader, device)
    print('Test accuracy: {}'.format(acc))
    return acc

In [9]:
def show_Network(Network):
  graphviz.set_jupyter_format('png')
  model_graph = draw_graph(Network, input_size=(1,3,32,32), expand_nested=True)
  model_graph.visual_graph

In [10]:
def FreezeLayers(Network, train_data_loader, test_data_loader, upd_layers):
  for params in Network.parameters():
    params.requires_grad = False
  Network.fc = torch.nn.Linear(Network.fc.in_features, upd_layers)
  return Network

In [11]:
def ModLastLAyers(Network, train_data_loader, test_data_loader, upd_layers):
    for params in Network.parameters():
     params.requires_grad = False
    Network.fc = torch.nn.Sequential(torch.nn.Linear(Network.fc.in_features, Network.fc.in_features//2), torch.nn.ReLU(), torch.nn.Linear(Network.fc.in_features//2, 10),)
    return Network

In [12]:
def get_first_ex(Network_type, weights):
  Network = Network_type(weights)
  transforms = weights.transforms()
  train_data_loader, test_data_loader = get_data_loader(transforms)
  Network.to(device)
  Network = FreezeLayers(Network, train_data_loader, test_data_loader, num_classes)
  Network.to(device)
  optimizer = torch.optim.Adam(Network.parameters(), lr = learning_rate)
  acc = train(Network, train_data_loader, test_data_loader, optimizer)
  print(Network)
  show_Network(Network)
  return acc

In [13]:
def get_second_ex(Network_type, weights):
  Network = Network_type(weights)
  transforms = weights.transforms()
  train_data_loader, test_data_loader = get_data_loader(transforms)
  Network = ModLastLAyers(Network, train_data_loader, test_data_loader, num_classes)
  Network.to(device)
  optimizer = torch.optim.Adam(Network.parameters(), lr = learning_rate)
  acc = train(Network, train_data_loader, test_data_loader, optimizer)
  print(Network)
  show_Network(Network)
  return acc

In [14]:
def show_exp_result(acc_array_, type_array_):
  results = {}
  for i in range(len(acc_array_)):
    results[type_array[i]] = acc_array[i].cpu()
  print(results)
  fig = plot.figure(figsize = (25, 6))
  plot.bar(results.keys(), results.values())
  plot.xlabel("Models experiments")
  plot.ylabel("Test accuracy")
  plot.title("Comparison of architectures modifications")
  for i in range(len(acc_array_)):
        plot.text(i, acc_array[i].cpu() + 0.01, f'{int(acc_array[i].cpu() * 100)} %', ha='center')
  plot.show()

In [15]:
acc_array.append(get_first_ex(torch_model.googlenet, torch_model.GoogLeNet_Weights.DEFAULT))

Files already downloaded and verified
Files already downloaded and verified
Epoch[0]: loss = 1.0425617694854736, accuracy = 0.7436400055885315, time = 1852.2726333141327
Epoch[1]: loss = 0.7531731128692627, accuracy = 0.7563599944114685, time = 1814.8770015239716
Epoch[2]: loss = 0.7123542428016663, accuracy = 0.7649999856948853, time = 1813.3419983386993
Epoch[3]: loss = 0.6962552070617676, accuracy = 0.767520010471344, time = 1811.1569983959198
Epoch[4]: loss = 0.6813787817955017, accuracy = 0.7729399800300598, time = 1813.7470004558563
Test accuracy: 0.7520999908447266
GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNo

In [16]:
acc_array.append(get_second_ex(torch_model.googlenet, torch_model.GoogLeNet_Weights.DEFAULT))

Files already downloaded and verified
Files already downloaded and verified
Epoch[0]: loss = 0.8613876700401306, accuracy = 0.7465800046920776, time = 1803.5559809207916
Epoch[1]: loss = 0.7187766432762146, accuracy = 0.7595000267028809, time = 1801.4439992904663
Epoch[2]: loss = 0.6918441653251648, accuracy = 0.7692999839782715, time = 1838.432424068451
Epoch[3]: loss = 0.6657810211181641, accuracy = 0.7740200161933899, time = 1863.7298049926758
Epoch[4]: loss = 0.6410001516342163, accuracy = 0.7800999879837036, time = 1802.3099970817566
Test accuracy: 0.7526999711990356
GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNo

In [17]:
acc_array.append(get_first_ex(torch_model.resnet50, torch_model.ResNet50_Weights.DEFAULT))

Files already downloaded and verified
Files already downloaded and verified
Epoch[0]: loss = 0.927531361579895, accuracy = 0.7813000082969666, time = 3023.5520520210266
Epoch[1]: loss = 0.6384558081626892, accuracy = 0.8062800168991089, time = 3031.619999885559
Epoch[2]: loss = 0.5744967460632324, accuracy = 0.8223400115966797, time = 3025.5650057792664
Epoch[3]: loss = 0.538637101650238, accuracy = 0.8285999894142151, time = 3319.984260082245
Epoch[4]: loss = 0.5147377848625183, accuracy = 0.8357800245285034, time = 3147.3574357032776
Test accuracy: 0.8066999912261963
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [18]:
acc_array.append(get_second_ex(torch_model.resnet50, torch_model.ResNet50_Weights.DEFAULT))

Files already downloaded and verified
Files already downloaded and verified
Epoch[0]: loss = 0.7186640501022339, accuracy = 0.8159199953079224, time = 3346.1530270576477
Epoch[1]: loss = 0.5339586734771729, accuracy = 0.8295000195503235, time = 3117.6498029232025
Epoch[2]: loss = 0.47789832949638367, accuracy = 0.8601800203323364, time = 3136.6504542827606
Epoch[3]: loss = 0.4155281186103821, accuracy = 0.8769400119781494, time = 3041.9669988155365
Epoch[4]: loss = 0.35352227091789246, accuracy = 0.9055399894714355, time = 3042.1560006141663
Test accuracy: 0.8159999847412109
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=F

In [1]:
acc_array.append(get_first_ex(torch_model.resnet34, torch_model.ResNet34_Weights.DEFAULT))

NameError: name 'acc_array' is not defined

In [16]:
acc_array.append(get_second_ex(torch_model.resnet34, torch_model.ResNet34_Weights.DEFAULT))

Files already downloaded and verified
Files already downloaded and verified
Epoch[0]: loss = 0.7807196378707886, accuracy = 0.7825599908828735, time = 1651.4454429149628


KeyboardInterrupt: 

In [15]:
acc_array.append(get_first_ex(torch_model.shufflenet_v2_x2_0, torch_model.ShuffleNet_V2_X2_0_Weights.DEFAULT))

C:\Users\soup4\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified
Epoch[0]: loss = 0.7855721712112427, accuracy = 0.8235200047492981, time = 868.353884935379


KeyboardInterrupt: 

In [ ]:
acc_array.append(get_second_ex(torch_model.shufflenet_v2_x2_0, torch_model.ShuffleNet_V2_X2_0_Weights.DEFAULT))

In [ ]:
acc_array

In [ ]:
type_array = ("GoogleNet First", "GoogleNet Second", "ResNet50 First", "ResNet50 Second", "ResNet34 First", "ResNet34 Second", "Shufflenet_v2_x2_0 First", "Shufflenet_v2_x2_0 Second")


In [ ]:
show_exp_result(acc_array, type_array)